In [86]:
import numpy as np
import pandas as pd
import itertools

In [87]:
import pandas as pd
df = pd.read_csv('data/similar.csv', sep = ',')
df.set_index('userID/game',inplace=True)
df_jacc = df.mask(df > 0, 1).fillna(0)
display(df)
display(df_jacc)

,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,3.0,3.0,4.0,NaN,4.0,2.0,3.0,NaN
mike0702,3.0,5.0,4.0,3.0,3.0,NaN,NaN,4.0
zephyros,NaN,4.0,NaN,5.0,NaN,NaN,2.0,1.0
dadvador,2.0,NaN,NaN,4.0,NaN,4.0,4.0,5.0


,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
mike0702,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
zephyros,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
dadvador,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


In [90]:
#print(df_jacc.loc['firechicken'])
def compute_similarity(df):
    result = []
    for item in list(itertools.combinations(df.index, 2)):
        a = df.loc[item[0]].values
        b = df.loc[item[1]].values
        intersection = np.sum(a * b)
        union = np.sum(a + b > 0)
        jacc_sim = intersection/union
        cosine_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
        jacc_dist = 1-jacc_sim
        cosine_dist = 1-cosine_sim
        result.append({"user1":item[0],
                       "user2":item[1],
                       'jaccard_similarity':jacc_sim,
                       'cosine_similarity':cosine_sim,
                       "jaccard_distance":jacc_dist,
                       "cosine_distance":cosine_dist})
    df_result = pd.DataFrame(result)

    return df_result
df_result = compute_similarity(df_jacc)
display(df_result)

InvalidIndexError: ('firechicken', slice(None, None, None))

In [85]:
good_bad = df.mask(df < 3, 0).fillna(0)
good_bad = good_bad.mask(good_bad > 0, 1)
display(df)
display(good_bad)
df_result = compute_similarity(good_bad)
display(df_result)

,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,3.0,3.0,4.0,NaN,4.0,2.0,3.0,NaN
mike0702,3.0,5.0,4.0,3.0,3.0,NaN,NaN,4.0
zephyros,NaN,4.0,NaN,5.0,NaN,NaN,2.0,1.0
dadvador,2.0,NaN,NaN,4.0,NaN,4.0,4.0,5.0


,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
mike0702,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
zephyros,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
dadvador,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


,user1,user2,jaccard_similarity,cosine_similarity,jaccard_distance,cosine_distance
0,firechicken,mike0702,0.571429,0.730297,0.428571,0.269703
1,firechicken,zephyros,0.166667,0.316228,0.833333,0.683772
2,firechicken,dadvador,0.125000,0.223607,0.875000,0.776393
3,mike0702,zephyros,0.333333,0.577350,0.666667,0.422650
4,mike0702,dadvador,0.250000,0.408248,0.750000,0.591752
5,zephyros,dadvador,0.200000,0.353553,0.800000,0.646447


In [82]:
ratings = np.array([[3, 3, 4, 0, 4, 2, 3, 0],
                    [3, 5, 4, 3, 3, 0, 0, 4],
                    [0, 4, 0, 5, 0, 0, 2, 1],
                    [2, 0, 0, 4, 0, 4, 4, 5]])
Ub = (ratings>0).astype(int)
print(Ub)

def jaccard(a,b):
    return (a*b).sum()/((a+b)>0).sum()

users = ["firechicken","mike0702","zephyros","dadvador"]

simmat=np.zeros((4,4))
for i in range(4):
    for j in range(4):
        simmat[i,j] = jaccard(Ub[i],Ub[j])
        if i<j:
            print(users[i]+'-'+users[j], jaccard(Ub[i],Ub[j]))

print(simmat)

[[1 1 1 0 1 1 1 0]
 [1 1 1 1 1 0 0 1]
 [0 1 0 1 0 0 1 1]
 [1 0 0 1 0 1 1 1]]
firechicken-mike0702 0.5
firechicken-zephyros 0.25
firechicken-dadvador 0.375
mike0702-zephyros 0.42857142857142855
mike0702-dadvador 0.375
zephyros-dadvador 0.5
[[1.         0.5        0.25       0.375     ]
 [0.5        1.         0.42857143 0.375     ]
 [0.25       0.42857143 1.         0.5       ]
 [0.375      0.375      0.5        1.        ]]


In [ ]:
def cos(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

simmat=np.zeros((4,4))
for i in range(4):
    for j in range(4):
        simmat[i,j] = cos(Ub[i],Ub[j])
        if i<j:
            print(users[i]+'-'+users[j], cos(Ub[i],Ub[j]))
print(simmat)

In [ ]:
Ur = (ratings>=3).astype(int)
print(Ur)

simmat=np.zeros((4,4))
for i in range(4):
    for j in range(4):
        simmat[i,j] = jaccard(Ur[i],Ur[j])
        if i<j:
            print(users[i]+'-'+users[j], jaccard(Ur[i],Ur[j]))